## 1. Setup

### 1.1. Libraries and Helper Functions

This notebook requires the TensorFlow dataset and other prerequisites that you must download. 

In [1]:
#@title Installs

# !pip install pydot --quiet
# !pip install gensim==3.8.3 --quiet
# !pip install tensorflow-datasets --quiet
# !pip install -U tensorflow-text==2.8.2 --quiet
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 114.9 MB/s eta 0:00:00


In [2]:
import torch

# Check if a GPU is available, otherwise use CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda')

In [4]:
!pip install tqdm boto3 requests regex sentencepiece sacremoses --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 58.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 53.4 MB/s eta 0:00:00


In [5]:
#@title Imports

# import numpy as np
# import tensorflow as tf
# from tensorflow import keras

# from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
# from tensorflow.keras.models import Model
# import tensorflow.keras.backend as K
# import tensorflow_datasets as tfds
# import tensorflow_text as tf_text

# from transformers import BertTokenizer, TFBertModel


# import sklearn as sk
# import os
# import nltk
# from nltk.data import find

# import matplotlib.pyplot as plt

# import re

# #This continues to work with gensim 3.8.3.  It doesn't yet work with 4.x.  
# #Make sure your pip install command specifies gensim==3.8.3
# import gensim


Below is a helper function to plot histories.

In [6]:
#@title Plotting Function

# # 4-window plot. Small modification from matplotlib examples.

# def make_plot(axs,
#               model_history1, 
#               model_history2, 
#               model_1_name='model 1',
#               model_2_name='model 2',
#               ):
#     box = dict(facecolor='yellow', pad=5, alpha=0.2)

#     for i, metric in enumerate(['loss', 'accuracy']):
#         # small adjustment to account for the 2 accuracy measures in the Weighted Averging Model with Attention
#         if 'classification_%s' % metric in model_history2.history:
#             metric2 = 'classification_%s' % metric
#         else:
#             metric2 = metric
        
#         y_lim_lower1 = np.min(model_history1.history[metric])
#         y_lim_lower2 = np.min(model_history2.history[metric2])
#         y_lim_lower = min(y_lim_lower1, y_lim_lower2) * 0.9

#         y_lim_upper1 = np.max(model_history1.history[metric])
#         y_lim_upper2 = np.max(model_history2.history[metric2])
#         y_lim_upper = max(y_lim_upper1, y_lim_upper2) * 1.1

#         for j, model_history in enumerate([model_history1, model_history2]):
#             model_name = [model_1_name, model_2_name][j]
#             model_metric = [metric, metric2][j]
#             ax1 = axs[i, j]
#             ax1.plot(model_history.history[model_metric])
#             ax1.plot(model_history.history['val_%s' % model_metric])
#             ax1.set_title('%s - %s' % (metric, model_name))
#             ax1.set_ylabel(metric, bbox=box)
#             ax1.set_ylim(y_lim_lower, y_lim_upper)

In [7]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [8]:
import pandas as pd

!ls "/content/drive/MyDrive/Colab Notebooks/bias_data"

bias_data  __MACOSX


In [9]:
train_path='/content/drive/MyDrive/Colab Notebooks/bias_data/bias_data/WNC/biased.word.train'
dev_path='/content/drive/MyDrive/Colab Notebooks/bias_data/bias_data/WNC/biased.word.dev'
test_path='/content/drive/MyDrive/Colab Notebooks/bias_data/bias_data/WNC/biased.word.test'

In [10]:
train = pd.read_csv(train_path, sep='\t', header=None)
dev = pd.read_csv(dev_path, sep='\t', header=None)
test = pd.read_csv(test_path, sep='\t', header=None)

In [11]:
print("Train length: {}".format(len(train)))
print("Dev length: {}".format(len(dev)))
print("Test length: {}".format(len(test)))

Train length: 53803
Dev length: 700
Test length: 1000


In [12]:
display(train)

,0,1,2,3,4,5,6
0,165188319,"ch ##lor ##of ##or ##m "" the molecular life ##...","ch ##lor ##of ##or ##m "" the molecular life ##...","chloroform ""the molecular lifesaver"" an articl...","chloroform ""the molecular lifesaver"" an articl...",NOUN NOUN NOUN NOUN NOUN PUNCT DET ADJ NOUN NO...,ROOT ROOT ROOT ROOT ROOT punct det amod dobj d...
1,123204846,the free software gnu class ##path project is ...,the free software gnu class ##path project is ...,the free software gnu classpath project is onl...,the free software gnu classpath project is par...,DET ADJ NOUN NOUN NOUN NOUN NOUN VERB ADV ADV ...,det amod nmod compound compound compound nsubj...
2,706783956,"other campaign ##ers , especially the controve...","other campaign ##ers , especially the british ...","other campaigners, especially the controversia...","other campaigners, especially the british acti...",ADJ NOUN NOUN PUNCT ADV DET ADJ ADJ NOUN ADJ N...,amod nsubj nsubj punct advmod det amod amod am...
3,612378448,vocalist rob half ##ord ' s performance is con...,vocalist rob half ##ord ' s performance is con...,vocalist rob halford's performance is consider...,vocalist rob halford's performance is consider...,ADJ X NOUN NOUN PUNCT PART NOUN VERB VERB NUM ...,amod amod poss poss punct case nsubjpass auxpa...
4,876796337,the proud general is a chinese animated featur...,the proud general is a chinese animated featur...,the proud general is a chinese animated featur...,the proud general is a chinese animated featur...,DET ADJ NOUN VERB DET ADJ VERB NOUN NOUN VERB ...,det amod nsubj ROOT det amod amod attr attr ac...
...,...,...,...,...,...,...,...
53798,341593940,the national lawyers guild is a progressive / ...,the national lawyers guild is a progressive ba...,the national lawyers guild is a progressive /l...,the national lawyers guild is a progressive ba...,DET ADJ NOUN ADJ VERB DET ADJ SYM ADJ PUNCT NO...,det amod compound nsubj ROOT det amod punct am...
53799,640510650,a plan to red ##eve ##lo ##p the old tiger sta...,a plan to red ##eve ##lo ##p the old tiger sta...,a plan to redevelop the old tiger stadium site...,a plan to redevelop the old tiger stadium site...,DET NOUN PART VERB VERB VERB VERB DET ADJ NOUN...,det nsubj aux acl acl acl acl det amod compoun...
53800,162719260,"instrumental ##ly , life ##son is regarded as ...","instrumental ##ly , life ##son is regarded as ...","instrumentally, lifeson is regarded as a virtu...","instrumentally, lifeson is regarded as a guita...",ADV ADV PUNCT NOUN NOUN VERB VERB ADP DET ADJ ...,advmod advmod punct nsubjpass nsubjpass auxpas...
53801,62331672,fly ##nt joined the us army in 1958 at only fi...,fly ##nt joined the us army in 1958 at only fi...,flynt joined the us army in 1958 at only fifte...,flynt joined the us army in 1958 at only fifte...,NOUN NOUN VERB DET PRON NOUN ADP NUM ADP ADV N...,nsubj nsubj ROOT det compound dobj prep pobj p...


In [13]:
def read_process_data(train_path, dev_path):
    train = pd.read_csv(train_path, sep='\t', header=None)
    dev = pd.read_csv(dev_path, sep = '\t', header=None)
    
    print("Train length: {}".format(len(train)))
    print("Dev length: {}".format(len(dev)))
    
    #Rename columns
    train.columns = ['id', 'annot_old', 'annot_new', 'biased', 'unbiased', 'tags', 'roots']
    dev.columns = ['id', 'annot_old', 'annot_new', 'biased', 'unbiased', 'tags', 'roots']
    
    #Process and recombine training data: 
    train_biased, dev_biased = pd.DataFrame(train.biased), pd.DataFrame(dev.biased)
    train_unbiased, dev_unbiased = pd.DataFrame(train.unbiased), pd.DataFrame(dev.unbiased)
    
    
    train_biased['label'], dev_biased['label'] = [1]*len(train_biased), [1]*len(dev_biased)
    train_unbiased['label'], dev_unbiased['label'] = [0]*len(train_unbiased), [0]*len(dev_unbiased)

    #Fix colnames
    train_biased.columns, dev_biased.columns = ['text', 'label'], ['text', 'label']
    train_unbiased.columns, dev_unbiased.columns = ['text', 'label'], ['text', 'label']
    #Combine
    train_all, dev_all = pd.concat([train_biased, train_unbiased]), pd.concat([dev_biased, dev_unbiased])
    
    return train_all, dev_all

In [14]:
train, dev = read_process_data(train_path, dev_path)

Train length: 53803
Dev length: 700


In [15]:
display(train)

,text,label
0,"chloroform ""the molecular lifesaver"" an articl...",1
1,the free software gnu classpath project is onl...,1
2,"other campaigners, especially the controversia...",1
3,vocalist rob halford's performance is consider...,1
4,the proud general is a chinese animated featur...,1
...,...,...
53798,the national lawyers guild is a progressive ba...,0
53799,a plan to redevelop the old tiger stadium site...,0
53800,"instrumentally, lifeson is regarded as a guita...",0
53801,flynt joined the us army in 1958 at only fifte...,0


In [16]:
train['text'].str.len().max()

1374

In [17]:
train['text'].str.len().min()

12

In [18]:
train['text'].str.len().mean()

162.60397189747783

In [19]:
display(dev)

,text,label
0,in addition to sponsoring palestinian terror a...,1
1,the game is currently played in 47 countries w...,1
2,no part of the valley lies in the area current...,1
3,scholars perceived that it was discordant with...,1
4,"since the chinese civil war in 1949, taiwan ha...",1
...,...,...
695,in 2008 five pharmaceutical companies received...,0
696,"the palm, a steakhouse restaurant chain origin...",0
697,d.c. united's early successes,0
698,on 29 june 2007 price gave birth to her third ...,0


The Wikipedia neutrality corpus is a collection of text samples from Wikipedia that have been labeled as either neutral or non-neutral. Exploratory Data Analysis (EDA) can provide insights into the characteristics of the corpus, such as the distribution of neutral and non-neutral samples, the length of the samples, and the most common words and phrases in each class. Here are some EDA techniques that we will apply to the Wikipedia neutrality corpus:

Data Cleaning: The first step in any EDA process is to clean the data. This might involve removing any extraneous metadata or formatting, and standardizing the text to a consistent format. It is also important to remove any samples that are incomplete or have missing labels.

Distribution of Labels: One important aspect of the corpus is the distribution of neutral and non-neutral samples. It is important to understand the balance of the corpus, as this can impact the performance of any machine learning models trained on it.

Word Frequencies: To understand the characteristics of the text in the corpus, it is useful to examine the most frequent words and phrases in each class. This can provide insights into the language and terminology used in neutral and non-neutral text.

Text Length: The length of the text samples can also provide useful insights into the corpus. For example, it might be interesting to compare the average length of neutral and non-neutral samples, or to look for patterns in the distribution of text lengths.


In [ ]:
# Work in progress
# import torch
# import torchtext

# def text_word_frequency(data_path):
#   with open(data_path, 'r') as f:
#             for line in f:
#                 line = line.strip()
#                 if line:
#                     # print(line)
#                     parts = line.split('\t')
#                     text = parts[0]
#                   # # Load the text data
#                   # text = "This is a sample text. This text can be used to demonstrate word frequency."
#                   tokens = text.lower().split()

#                   # Create a Torchtext vocabulary from the text
#                   vocab = torchtext.vocab.Vocab(
#                       counter=torchtext.vocab.Counter(tokens),
#                       specials=['<pad>', '<unk>']
#                   )

#                   # Create a tensor of token indices for the text
#                   token_indices = torch.tensor([vocab[token] for token in tokens])

#                   # Count the frequency of each word
#                   word_frequencies = torch.bincount(token_indices)

#                   # Print the most common words and their frequency
#                   for i in torch.argsort(word_frequencies, descending=True)[:10]:
#                       print(vocab.itos[i], word_frequencies[i].item())

Below code is implementation of BART (Bidirectional and Auto-Regressive Transformer) model for sequence-to-sequence (seq2seq) tasks using PyTorch. The BART model is a variant of the Transformer model and is pre-trained on a large corpus of text data using a denoising autoencoder objective. It is capable of generating high-quality text with coherent sentence structures.

The code defines a PyTorch Dataset class MyDataset that reads the input and target text sequences from a text file and tokenizes them using the BART tokenizer. It then defines the collate_fn function to pad the sequences in each batch to the same length. The code then defines the BartEncoderDecoder class, which is the main model class that uses the BART encoder and decoder architecture.

The code instantiates the BART tokenizer and the BartEncoderDecoder model, and loads the training and validation data using the MyDataset class. It also defines the optimizer and loss function. The code then runs the training loop for a specified number of epochs, during which it trains the model on the training set and evaluates it on the validation set.

During training, the code iterates over the batches of training data and performs forward and backward passes through the model to compute the loss and update the model parameters using the Adam optimizer. The code also implements gradient accumulation by accumulating the gradients over a specified number of batches to reduce the memory requirements during training. After training on all the batches, the code computes the average loss over the entire training set and evaluates the model on the validation set.

During validation, the code iterates over the batches of validation data and generates the predicted text sequences using the generate method of the BART decoder. It then computes the loss between the predicted and target sequences and accumulates the loss over all the batches to compute the average loss over the entire validation set. Finally, the code prints the average training and validation loss for each epoch.

In [20]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from transformers import BartTokenizer, BartConfig, BartForConditionalGeneration,BartModel
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F
# the MyDataset class takes three arguments:

# data_path: The path to the training data file.
# tokenizer: The tokenizer used to preprocess the input data.
# max_seq_len: The maximum sequence length allowed for the input sequences.
# In the __init__ method, the training data is loaded from the specified file, and each line of text is stored in a list.

# In the __len__ method, the length of the dataset is returned (i.e., the number of lines in the training data file).

# In the __getitem__ method, an individual training example is retrieved based on the specified index. The input text is tokenized using the specified tokenizer and truncated to the specified maximum sequence length, and the resulting token ids are converted to a PyTorch tensor and returned as the training example.


class MyDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_seq_len):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

        self.source_ids = []
        self.target_ids = []

        with open(data_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line:
                    # print(line)
                    parts = line.split('\t')
                    source_seq = parts[0]
                    target_seq = parts[1]

                    #source_seq, target_seq = line.split('\t')
                    source_ids = tokenizer.encode(
                        source_seq, add_special_tokens=True, truncation=True,
                        max_length=max_seq_len
                    )
                    target_ids = tokenizer.encode(
                        target_seq, add_special_tokens=True, truncation=True,
                        max_length=max_seq_len
                    )
                    self.source_ids.append(source_ids)
                    self.target_ids.append(target_ids)

    def __len__(self):
        return len(self.source_ids)

    def __getitem__(self, index):
        source_ids = self.source_ids[index]
        target_ids = self.target_ids[index]

        # Create a PyTorch tensor from the tokenized sequences
        source_ids = torch.tensor(source_ids)
        target_ids = torch.tensor(target_ids)

        return {'source_ids': source_ids, 'target_ids': target_ids}

    def collate_fn(self, batch):
        # Pad sequences in batch to have the same length
        source_ids = pad_sequence([item['source_ids'] for item in batch], batch_first=True, padding_value=self.tokenizer.pad_token_id)
        target_ids = pad_sequence([item['target_ids'] for item in batch], batch_first=True, padding_value=self.tokenizer.pad_token_id)

        return {'source_ids': source_ids, 'target_ids': target_ids}


In [21]:
# Define hyperparameters
BATCH_SIZE = 16
NUM_EPOCHS = 10
LEARNING_RATE = 1e-4
MAX_SEQ_LEN = 128
D_MODEL = 768
NUM_HEADS = 12
NUM_ENCODER_LAYERS = 6
NUM_DECODER_LAYERS = 6
DIM_FEEDFORWARD = 3072
DROPOUT = 0.1
GRADIENT_ACCUMULATION_STEPS = 4  # accumulate gradients over 4 batches
# Instantiate BART tokenizer
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

# Define BART encoder-decoder model
class BartEncoderDecoder(nn.Module):
    def __init__(self, bart_config):
        super().__init__()
        self.encoder = BartModel(bart_config)
        self.decoder = BartForConditionalGeneration(bart_config)

    def forward(self, input_ids, decoder_input_ids):
        encoder_outputs = self.encoder(input_ids=input_ids)
        decoder_outputs = self.decoder(input_ids=decoder_input_ids,encoder_outputs=encoder_outputs)
        return decoder_outputs

# Instantiate BART encoder-decoder model
bart_config = BartConfig(d_model=D_MODEL, encoder_layers=NUM_ENCODER_LAYERS, decoder_layers=NUM_DECODER_LAYERS, 
                         encoder_attention_heads=NUM_HEADS, decoder_attention_heads=NUM_HEADS,
                         encoder_ffn_dim=DIM_FEEDFORWARD, decoder_ffn_dim=DIM_FEEDFORWARD, dropout=DROPOUT)
bart_model = BartEncoderDecoder(bart_config).cuda() # move the model to CUDA

# Load training and validation data
train_data = MyDataset(train_path, tokenizer, MAX_SEQ_LEN)
val_data = MyDataset(dev_path, tokenizer, MAX_SEQ_LEN)

# Instantiate data loaders
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,collate_fn=train_data.collate_fn)
val_loader = DataLoader(val_data, batch_size=BATCH_SIZE,collate_fn=val_data.collate_fn)

# Instantiate optimizer and loss function
optimizer = Adam(bart_model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss().cuda() # move the loss function to CUDA

# Training loop
for epoch in range(NUM_EPOCHS):
    # Train the model
    bart_model.train()
    train_loss = 0
    for i, batch in enumerate(train_loader):
        src = batch['source_ids'].cuda()#to(device)
        tgt = batch['target_ids'].cuda() #to(device)
        optimizer.zero_grad()
        outputs = bart_model(input_ids=src, decoder_input_ids=tgt[:, :-1].cuda()) #.to(device)
        output_shape = outputs.logits.shape
        # print(output_shape)
        outputs = outputs[0].view(-1, tokenizer.vocab_size)
        loss = criterion(outputs.reshape(-1, tokenizer.vocab_size), tgt[:, 1:].reshape(-1))
        loss /= GRADIENT_ACCUMULATION_STEPS  # divide loss by number of steps to average gradients
        loss.backward()
        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            optimizer.zero_grad()
        train_loss += loss.item()
        # loss = criterion(outputs.logits.reshape(-1, tokenizer.vocab_size), tgt[:, 1:].reshape(-1))
        # loss.backward()
        # optimizer.step()
        # train_loss += loss.item()
    train_loss /= len(train_loader)



KeyboardInterrupt: ignored

In [50]:
    # Evaluate the model on the validation set
    bart_model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            src = batch['source_ids'].cuda()#.to(device)
            tgt = batch['target_ids'].cuda()#.to(device)
            
            outputs = bart_model.decoder.generate(input_ids=src, max_length=MAX_SEQ_LEN, num_beams=5)
            generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            gold_text = tokenizer.batch_decode(tgt[:, 1:], skip_special_tokens=True)
            print(outputs.shape)
            outputs = outputs[0].view(-1, outputs.shape[-1]).float()
            print(outputs.shape)
            outputs = F.log_softmax(outputs, dim=-1)
            # tgt = tgt[:, 1:].reshape(-1)
            # tgt = tgt[:, 1:].flatten()
            # tgt = tgt.flatten()#[:, 1:].reshape(-1)
            print(tgt.shape)
            # print((outputs.squeeze(0)).shape)
            loss = criterion(outputs.reshape(-1, outputs.shape[-1]), tgt[:, 1:]) #.reshape(-1)
            # loss = criterion(outputs.squeeze(0), tgt) #tgt[:, 1:].reshape(-1,1)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    # Print training and validation
    print(f'Epoch {epoch + 1}/{NUM_EPOCHS}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}')

torch.Size([16, 128])
torch.Size([1, 128])
torch.Size([16, 79])


ValueError: ignored

In [40]:
# bart_model.eval()
# val_loss = 0
# with torch.no_grad():
#     for batch in val_loader:
#         src = batch['source_ids'].cuda()
#         tgt = batch['target_ids'].cuda()

#         outputs = bart_model.decoder.generate(input_ids=src, max_length=MAX_SEQ_LEN, num_beams=5)
#         generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#         gold_text = tokenizer.batch_decode(tgt[:, 1:], skip_special_tokens=True)

#         # Reshape the tensors to match
#         outputs = outputs[0].view(-1, outputs.shape[-1]).float()
#         outputs = F.log_softmax(outputs, dim=-1)
#         tgt = tgt[:, 1:].flatten()

#         outputs = F.log_softmax(outputs, dim=-1)
#         loss = criterion(outputs, tgt)
#         val_loss += loss.item()

# val_loss /= len(val_loader)

# # Print training and validation
# print(f'Epoch {epoch + 1}/{NUM_EPOCHS}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}')


ValueError: ignored

In [35]:
# # Evaluate the model on the validation set
# bart_model.eval()
# val_loss = 0
# with torch.no_grad():
#     for batch in val_loader:
#         src = batch['source_ids'].cuda()
#         tgt = batch['target_ids'].cuda()

#         # print(src)
#         # print(tgt)
#         outputs = bart_model.decoder.generate(input_ids=src, max_length=MAX_SEQ_LEN, num_beams=5)
#         generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
#         gold_text = tokenizer.batch_decode(tgt[:, 1:], skip_special_tokens=True)

#         # Reshape the tensors to match
#         outputs = outputs[:, :tgt.shape[1]-1].reshape(-1, outputs.shape[-1])
#         tgt = tgt[:, 1:].reshape(-1)
        
#         print(outputs.shape)
#         print(tgt.shape)

#         outputs = F.log_softmax(outputs, dim=-1)
#         loss = criterion(outputs, tgt)
#         val_loss += loss.item()

# val_loss /= len(val_loader)

# # Print training and validation
# print(f'Epoch {epoch + 1}/{NUM_EPOCHS}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}')


tensor([[    0, 24133,  5479,   698,     2,     1],
        [    0, 35868,  3118,  3414,     2,     1],
        [    0,   245, 24990, 37664,  2890,     2],
        [    0,  6232,  2890, 11670,  3118,     2],
        [    0, 29187,  1366,  5677,   612,     2],
        [    0, 11343, 41258,   466,  3416,     2],
        [    0, 38613,  4718,  1244,  3761,     2],
        [    0,  3367, 34248,  3706,  3083,     2],
        [    0, 40944,  1366,  4531,  1549,     2],
        [    0, 38714,  3414,  1646,  4429,     2],
        [    0,  4718, 12636,   406, 36192,     2],
        [    0, 40093,  5606,  1646,  2481,     2],
        [    0,   398, 35992,  5352,   996,     2],
        [    0,   996,   844,  3248, 40690,     2],
        [    0, 33333,  3546,  4563,  4718,     2],
        [    0, 31412,  2481,  6617,  1646,     2]], device='cuda:0')
tensor([[    0,   179,  1285,  ...,     1,     1,     1],
        [    0,   627,   177,  ...,     1,     1,     1],
        [    0,  2362,   233,  ...

RuntimeError: ignored